In [ ]:
!pip install -U transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 105.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2


In [ ]:
!pip install transformers sentencepiece sacremoses
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 48.7 MB/s eta 0:00:00


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("translation", model="facebook/nllb-200-distilled-1.3B")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from tqdm import tqdm
import os

model_name = "facebook/nllb-200-distilled-600M"
src_lang = "eng_Latn"
tgt_lang = "yor_Latn"

dev_file = "https://raw.githubusercontent.com/Tonative-Research/Extending-AfriXNLI/refs/heads/main/dev/xnli.dev%20-%20eng_dataset.csv"
test_file = "https://raw.githubusercontent.com/Tonative-Research/Extending-AfriXNLI/refs/heads/main/test/xnli.test%20-%20english.csv"

# Create directories
os.mkdir('dev')
sentence1_dev = 'dev/sentence1_translated.csv'
sentence2_dev = 'dev/sentence2_translated.csv'

os.mkdir('test')
sentence1_test = 'test/sentence1_translated.csv'
sentence2_test = 'test/sentence2_translated.csv'


In [ ]:
df = pd.read_csv(dev_file, sep=",")
df.head()

,language,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,promptID,pairID,genre,label1,label2,label3,label4,label5,sentence1_tokenized,sentence2_tokenized,match
0,en,neutral,NaN,NaN,NaN,NaN,"And he said, Mama, I'm home.",He called his mom as soon as the school bus dr...,1,1,facetoface,neutral,contradiction,neutral,neutral,neutral,"And he said , Mama , I 'm home .",He called his mom as soon as the school bus dr...,True
1,en,contradiction,NaN,NaN,NaN,NaN,"And he said, Mama, I'm home.",He didn't say a word.,1,2,facetoface,contradiction,contradiction,contradiction,contradiction,contradiction,"And he said , Mama , I 'm home .",He didn 't say a word .,True
2,en,entailment,NaN,NaN,NaN,NaN,"And he said, Mama, I'm home.",He told his mom he had gotten home.,1,3,facetoface,entailment,entailment,neutral,entailment,entailment,"And he said , Mama , I 'm home .",He told his mom he had gotten home .,True
3,en,neutral,NaN,NaN,NaN,NaN,I didn't know what I was going for or anything...,I have never been to Washington so when I was ...,6,16,facetoface,neutral,neutral,neutral,neutral,neutral,I didn 't know what I was going for or anythin...,I have never been to Washington so when I was ...,True
4,en,contradiction,NaN,NaN,NaN,NaN,I didn't know what I was going for or anything...,I knew exactly what I needed to do as I marche...,6,17,facetoface,contradiction,contradiction,contradiction,contradiction,contradiction,I didn 't know what I was going for or anythin...,I knew exactly what I needed to do as I marche...,True


In [ ]:
df.shape

(2490, 19)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang=src_lang)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

Loading model...


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
def translate_batch(texts, batch_size=8):
    translations = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
        batch = [str(t) if pd.notna(t) else "" for t in texts[i:i+batch_size]]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        translated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer._convert_token_to_id_with_added_voc(tgt_lang),
            max_length=512
        )
        batch_translations = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        translations.extend(batch_translations)
    return translations


In [ ]:

def process_file(input_path, output_path1, output_path2):
    print(f"Processing {input_path}...")
    df = pd.read_csv(input_path, sep=",")

    sentence1_translated_df = pd.DataFrame()
    sentence2_translated_df = pd.DataFrame()

    sentence1_translated_df[['promptID', 'sentence1']] = df[['promptID', 'sentence1']]
    sentence2_translated_df[['promptID', 'sentence2']] = df[['promptID', 'sentence2']]



    print("Translating sentence...")
    sentence1_translated_df["sentence1_translated"] = translate_batch(df["sentence1"].tolist(), batch_size=8)
    sentence2_translated_df["sentence2_translated"] = translate_batch(df["sentence2"].tolist(), batch_size=8)

    #sentences
    sentence1_translated_df = sentence1_translated_df[["promptID", "sentence1", "sentence1_translated"]]
    sentence2_translated_df = sentence2_translated_df[["promptID", "sentence2", "sentence2_translated"]]


    #duplicates
    sentence1_translated_df = sentence1_translated_df.drop_duplicates()
    sentence2_translated_df = sentence2_translated_df.drop_duplicates()

    #save
    sentence1_translated_df.to_csv(output_path1, index=False)
    sentence2_translated_df.to_csv(output_path2, index=False)

    print(f"Saved translated file {output_path1} and {output_path2}")

In [ ]:
process_file(dev_file, sentence1_dev, sentence2_dev)

Processing https://raw.githubusercontent.com/Tonative-Research/Extending-AfriXNLI/refs/heads/main/dev/xnli.dev%20-%20eng_dataset.csv...
Translating sentence...



Translating: 100%|██████████| 312/312 [03:33<00:00,  1.46it/s]

Saved translated file dev/sentence1_translated.csv and dev/sentence2_translated.csv


In [ ]:
process_file(test_file, sentence1_test, sentence2_test)

Processing https://raw.githubusercontent.com/Tonative-Research/Extending-AfriXNLI/refs/heads/main/test/xnli.test%20-%20english.csv...
Translating sentence...



Translating: 100%|██████████| 627/627 [08:06<00:00,  1.29it/s]

Saved translated file test/sentence1_translated.csv and test/sentence2_translated.csv


In [ ]:
kikuyu = pd.read_csv('/content/dev/sentence1_translated.csv')
kikuyu.head()

,promptID,sentence1,sentence1_translated
0,1,"And he said, Mama, I'm home.","Nake akiuga, ""Amai, ndĩ mũciĩ""."
1,6,I didn't know what I was going for or anything...,"Ndingĩamenyete kĩrĩa ngwendaga gwĩka, na nĩ ũn..."
2,8,He didn't get to go.,Ndaigana gũthiĩ.
3,9,"And I was like OK, and that was it!","Na niĩ ngĩthiaga na mĩthiĩre mĩega, na ũguo nĩ..."
4,11,I was just there just trying to figure it out.,Ndĩ kũu ngĩgeria gũtaũkĩrũo nĩ ũndũ ũcio.
